Для обхода блокировок при большом количестве запросов к апи была использована библиотека генерирующая User agent 
чтобы сервер не видел что мы используем питон и библиотеку requests для парсинга а так же для избежания блокировки по 
ip мы использовали TOR. К тому же апи HH не позволяет получать дальше чем двухтысячный результат запроса, для решения этой проблемы данные были кластеризованы по параметрам чтобы при каждом запросе у нас получался небольшой кластер(город, опыт работы и т.д).

In [1]:
import requests
import json
import itertools
import socks
import socket
import logging
import pandas as pd
import sys

from bs4 import BeautifulSoup
from fake_useragent import UserAgent

дефолтный порт TOR 9150.
Зададим при помощи библиотек socks и socket дефолтный порт через который будут ходить все запросы, следовательно
все запросы будут посылаться через TOR, который по умолчанию меняет цепочку через которую идут пакеты и соответственно
выходная нода, ip которой видит конечный сервер, тоже меняется

In [ ]:
logging.basicConfig(stream=sys.stdout)
logger = logging.Logger('parser')
logger.setLevel(logging.DEBUG)
socks.set_default_proxy(socks.SOCKS5, "localhost", 9150)
socket.socket = socks.socksocket

пример сгенерированного UserAgent

In [3]:
UserAgent().chrome

'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.67 Safari/537.36'

In [4]:
class Parser:
    def __init__(self,base_url='http://api.hh.ru/'):
        self.base_url = base_url
        self.parsed_vacancies = []
    
    def get_areas(self):
        res = []
        for i in json.loads(requests.get(self.base_url + 'areas').text)[0]['areas']:
            res.append((i['id'], i['name']))
        return res
    
    def get_vacancies(self):
        self.parsed_vacancies = []
        experience = ['between1And3', 'between3And6', 'noExperience', 'moreThan6']
        for area, area_name in self.get_areas():
            logger.warning('current area: {}'.format(area_name))
            for exp, ind in itertools.product(experience, [i for i in range(100)]):
                for i in range(20):
                    r = requests.get(self.base_url + 'vacancies', 
                                     params={'text': '', 'per_page': 100, 'page': i, 'area': area, 'industry': ind, 'experience': exp},
                                     headers={'User-Agent': UserAgent().chrome})
                    if not r.ok:
                        logger.warning('page error resp code: {}'.format(r.status_code))
                        logger.warning(r.text)
                    parsed = json.loads(r.text)
                    if parsed.get('items'):
                        self.parsed_vacancies.extend(parsed.get('items'))
                        logger.warning('parsed: {}'.format(len(parsed.get('items'))))
                    else:
                        break
                    if len(parsed.get('items')) < 100:
                        break
        return self.parsed_vacancies
    
    def to_pandas(self):
        conv = []
        for vacancy in self.parsed_vacancies:
            conv.append(
                [vacancy['area']['name'],
                 vacancy['employer']['name'],
                 vacancy['salary'],
                 vacancy['name'],
                 vacancy['snippet'].get('requirement', ''),
                 vacancy['snippet'].get('responsibility', '')]
            )
        return pd.DataFrame(data=conv, columns=['area', 'company_name', 'salary', 'name', 'req', 'resp'])



In [ ]:
parser = Parser()
parser.get_vacancies()
data = parser.to_pandas()